In [15]:
from pathlib import Path
from typing import Dict, Iterator, Optional, Any
import chess.pgn

In [16]:
def iter_games(pgn_path: Path) -> Iterator[chess.pgn.Game]:
    """Yield games one by one from a PGN file"""
    
    if pgn_path.suffix.lower() != ".pgn":
        raise ValueError(f"Expected a .pgn file, got: {pgn_path.suffix}")
    
    with open(pgn_path, "r", encoding="utf-8", errors="replace") as f:
        while True:
            game = chess.pgn.read_game(f)
            if game is None:
                break
            yield game

In [17]:
def pgn_to_db(chess_games_folder: Path, c, conn):
    """Read PGN files and insert FEN positions into the database"""       

    for pgn_file in chess_games_folder.iterdir():
        for game in iter_games(pgn_file):
            board = game.board()
            fen = board.fen()
            c.execute("INSERT OR IGNORE INTO positions (fen) VALUES (?)", (fen,))

            for move in game.mainline_moves():
                try:
                    board.push(move)
                    fen = board.fen()
                    c.execute("INSERT OR IGNORE INTO positions (fen) VALUES (?)", (fen,))
                except ValueError as e:
                    print(f"Skipping illegal move in {pgn_file}:{e}")            

    conn.commit()

    c.execute("SELECT COUNT(*) from positions")
    total = c.fetchone()[0]
    print(f"{total} positions loaded.")

In [18]:
import sqlite3

conn = sqlite3.connect("positions.db")

c = conn.cursor()

c.execute("""
          CREATE TABLE IF NOT EXISTS positions (
          id INTEGER PRIMARY KEY,
          fen TEXT UNIQUE NOT NULL,
          stockfish_score REAL,
          label TEXT
        )
""")

conn.commit()

chess_games_folder = Path("./chess_games_sample")
pgn_to_db(chess_games_folder, c, conn)

conn.close()

16835 positions loaded.


In [ ]:
# Todo: Connect Stockfish